In [5]:
import os
import sys
sys.path.append("../../")

# VI

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()

        self.f1 = nn.Linear(6, 128)
        self.f2 = nn.Linear(128, 128)
        self.f3 = nn.Linear(128, 128)
        self.f4 = nn.Linear(128, 1)
        
    def forward(self, pos,theta):
        x = torch.cat([pos,theta], dim=1)
        x = F.relu(self.f1(x))
        x = F.relu(self.f2(x))
        x = F.relu(self.f3(x))
        x = F.sigmoid(self.f4(x))
        return x

# Instantiate the model
obs_model = NN()

# Load the model
obs_model.load_state_dict(torch.load('obs_model_0.pth',map_location=torch.device('cpu') ,weights_only=True))

obs_model.eval()
obs_model(torch.tensor([0.5, 0.5]).unsqueeze(0), torch.tensor([0.5, 0.0, 0.0, 0.5]).unsqueeze(0))

tensor([[0.4451]], grad_fn=<SigmoidBackward0>)

In [7]:
from environment.env import POMDPDeformedGridworld
pomdp_env = POMDPDeformedGridworld()
pomdp_env.reset()

({'obs': tensor(0.), 'pos': tensor([0.5750, 0.3591])}, {})

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal

# # Define the model and likelihood
# def likelihood(y, y_pred, noise_std):
#     return Normal(y_pred, noise_std).log_prob(y).sum()

# Define the likelihood: Bernoulli(f(theta)), where f(theta) = sigmoid(theta)
def likelihood_exact(y,X, theta, ):
    """
    y: observation
    X: position of the agent
    theta: deformation parameter (sampled from the variational distribution)
    
    Returns the log likelihood of the observation y given the position X and the deformation parameter theta
    """
    if isinstance(X, list) or (isinstance(X, torch.Tensor) and X.shape[0] > 1):
        likelihood = 0
        for x,y,t in zip(X,y,theta):
            f_theta = torch.tensor([pomdp_env.batchedobserve(x.tolist(),t.reshape(2,2).tolist())]) #function of  X=(x,y) and theta sampled from variational distribution q_lambda
            likelihood += (y * torch.log(f_theta + 10e-9) + (1 - y) * torch.log(1 - f_theta + 10e-9))
        return likelihood

    f_theta = torch.tensor([pomdp_env.batchedobserve(X.squeeze().tolist(),theta.reshape(2,2).tolist())]) #function of  X=(x,y) and theta sampled from variational distribution q_lambda
    return y * torch.log(f_theta + 10e-9) + (1 - y) * torch.log(1 - f_theta + 10e-9)

def likelihood_model(y,X, theta, ):
    """
    y: observation
    X: position of the agent
    theta: deformation parameter (sampled from the variational distribution)
    
    Returns the log likelihood of the observation y given the position X and the deformation parameter theta
    """
    # print(X.shape, theta.shape)
    f_theta = obs_model(X, theta)
    return torch.distributions.Bernoulli(f_theta).log_prob(y).sum()
    # return (y * torch.log(f_theta + 10e-9) + (1 - y) * torch.log(1 - f_theta + 10e-9)).sum()

# Define the prior
def prior(theta):
    mu = torch.tensor([torch.mean(torch.tensor(pomdp_env.stretch_range)),
                    torch.mean(torch.tensor(pomdp_env.shear_range)),
                    torch.mean(torch.tensor(pomdp_env.shear_range)),
                    torch.mean(torch.tensor(pomdp_env.stretch_range))])
    sigma = torch.tensor([.5,.5,.5,.5])
    return Normal(mu, sigma).log_prob(theta).sum()

# Variational distribution (q_lambda)
class VariationalDistribution(nn.Module):
    def __init__(self, dim):
        super(VariationalDistribution, self).__init__()
        self.mu = nn.Parameter(torch.zeros(dim))
        self.log_sigma = nn.Parameter(torch.zeros(dim))
    
    def sample(self, num_samples=1):
        epsilon = torch.randn(num_samples, self.mu.size(0), device=self.mu.device)
        sigma = torch.exp(self.log_sigma)
        return self.mu + sigma * epsilon  # Reparameterization trick
    
    def log_prob(self, theta):
        sigma = torch.exp(self.log_sigma)
        return Normal(self.mu, sigma).log_prob(theta).sum()
    
    def entropy(self):
        sigma = torch.exp(self.log_sigma)
        return Normal(self.mu, sigma).entropy().sum()

# BBVI Training Loop
def bbvi(X, y, variational_dist, num_epochs=5000, num_samples=10, lr=0.01):
    optimizer = optim.Adam(variational_dist.parameters(), lr=lr)
    noise_std = 0.1  # Observation noise

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        
        # Monte Carlo approximation of ELBO
        elbo = 0
        for i in range(num_samples):
            # Sample theta ~ q_lambda
            theta = variational_dist.sample()#(X.shape[0]) 
            theta = theta.repeat(X.shape[0],1)
            # Log likelihood p(x|theta) theta is sampled from the variational distribution
            log_likelihood = likelihood_exact(y,X,theta)
            
            # Log prior
            log_prior = prior(theta)
            
            # Log variational density
            log_q = variational_dist.log_prob(theta)
            
            # ELBO term
            elbo += log_likelihood + log_prior - log_q
        
        elbo /= num_samples  # Average over samples
        
        # Maximize ELBO (minimize negative ELBO)
        loss = -elbo
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, ELBO: {elbo.item():.4f}")
            print(f"Variational entropy: {variational_dist.entropy().item():.4f}")
    return variational_dist



In [17]:
# Synthetic data for Bayesian linear regression
torch.manual_seed(42)

# X va sostituito con gli stati del pomdp (x,y)
pos = pomdp_env.get_state()['pos']
X = torch.tensor([pos[0], pos[1]]).unsqueeze(0) # one sample only
# X = torch.randn(100, 2)  # 100 samples, 2 features

# y va sostituito con le osservazioni del pomdp negli stati pos=(x.y)
y = pomdp_env.get_state()['obs']

# true_theta va sostituito con i parametri reali del pomdp
true_theta = pomdp_env.transformation_matrix #torch.tensor([1.5, 0.5])


# print(X.shape, true_theta.shape, y.shape)

# Initialize variational distribution
variational_dist = VariationalDistribution(dim=4)

# Train with BBVI
trained_dist = bbvi(X, y, variational_dist, num_epochs=100, num_samples=100, lr=0.05)

# Output trained variational parameters
print(f"Variational Mean: {trained_dist.mu.detach().numpy()}")
print(f"Variational Log Std: {trained_dist.log_sigma.detach().numpy()}")

print(f"True Theta: {true_theta}")

/tmp/ipykernel_46624/2130527995.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mu = torch.tensor(torch.tensor([torch.mean(torch.tensor(pomdp_env.stretch_range)),


Epoch 10, ELBO: -1.4618
Variational entropy: 3.8229
Epoch 20, ELBO: -2.0629
Variational entropy: 2.6913
Epoch 30, ELBO: -3.2017
Variational entropy: 2.3513
Epoch 40, ELBO: -1.8546
Variational entropy: 2.5077
Epoch 50, ELBO: -2.0608
Variational entropy: 2.8103
Epoch 60, ELBO: -2.5817
Variational entropy: 2.9983
Epoch 70, ELBO: -2.3840
Variational entropy: 2.9963
Epoch 80, ELBO: -2.2114
Variational entropy: 2.8595
Epoch 90, ELBO: -0.9313
Variational entropy: 2.7943
Epoch 100, ELBO: -2.0220
Variational entropy: 2.8118
Variational Mean: [0.69867116 0.00935323 0.02087384 0.72142774]
Variational Log Std: [-0.68619794 -0.7461779  -0.6965455  -0.7350372 ]
True Theta: [[0.9150712259141278, -0.026402282565070212], [-0.08528106287796525, 0.8148674518812786]]


# trajectory generation

In [19]:
pomdp_env.reset()
print(pomdp_env.transformation_matrix)
s = pomdp_env.get_state()
pos = [s['pos']]
obs = [s['obs']]
while True:
    try:
        state, _, _, _, _ = pomdp_env.step(int(input()))
        pos.append(state['pos'])
        obs.append(state['obs'])
    except:
        break

[[0.8109401012886684, 0.06654548799716531], [-0.020748753038606266, 0.8358242639626121]]


In [20]:
positions, observations = pos, obs

In [21]:
len(observations)

71

In [22]:
# X va sostituito con gli stati del pomdp (x,y)
X = torch.stack(positions)

# y va sostituito con le osservazioni del pomdp negli stati pos=(x.y)
y = torch.stack(observations)

# true_theta va sostituito con i parametri reali del pomdp
true_theta = pomdp_env.transformation_matrix 

# print(X.shape, true_theta.shape, y.shape)

# Initialize variational distribution
variational_dist = VariationalDistribution(dim=4)

# Train with BBVI
trained_dist = bbvi(X, y, variational_dist, num_epochs=100, num_samples=200, lr=0.1)

# Output trained variational parameters
print(f"Variational Mean: {trained_dist.mu.detach().numpy()}")
print(f"Variational Log Std: {trained_dist.log_sigma.detach().numpy()}")

print(f"True Theta: {true_theta}")

/tmp/ipykernel_46624/2130527995.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mu = torch.tensor(torch.tensor([torch.mean(torch.tensor(pomdp_env.stretch_range)),


Epoch 10, ELBO: -201.7300
Variational entropy: 2.5280
Epoch 20, ELBO: -203.0858
Variational entropy: 1.8568
Epoch 30, ELBO: -133.7054
Variational entropy: 2.6228
Epoch 40, ELBO: -174.8132
Variational entropy: 3.2287
Epoch 50, ELBO: -132.3981
Variational entropy: 2.9931
Epoch 60, ELBO: -147.2686
Variational entropy: 2.7454
Epoch 70, ELBO: -151.7767
Variational entropy: 2.8417
Epoch 80, ELBO: -149.1945
Variational entropy: 2.9733
Epoch 90, ELBO: -158.0813
Variational entropy: 2.9115
Epoch 100, ELBO: -156.1136
Variational entropy: 2.8544
Variational Mean: [ 0.6744745   0.01219801 -0.05089802  0.71386325]
Variational Log Std: [-0.72823495 -0.73788625 -0.6774991  -0.6777137 ]
True Theta: [[0.8109401012886684, 0.06654548799716531], [-0.020748753038606266, 0.8358242639626121]]


# new VI

In [71]:
import torch
import torch.nn as nn
import torch.distributions as dist
from torch.optim import Adam
import numpy as np

class VariationalBayesianInference:
    def __init__(self, f, input_dim, latent_dim=1, hidden_dim=32):
        """
        Initialize the variational Bayesian inference model.
        
        Args:
            f: callable, the known function linking X and y through theta
            input_dim: int, dimension of input X
            latent_dim: int, dimension of latent parameter theta
            hidden_dim: int, dimension of hidden layers in the neural network
        """
        self.f = f
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        
        # Variational parameters (mean and log variance of q(theta))
        self.q_mu = nn.Parameter(torch.randn(latent_dim))
        self.q_logvar = nn.Parameter(torch.zeros(latent_dim))
        
        # Prior parameters (assumed to be standard normal)
        # self.prior = dist.Normal(torch.zeros(latent_dim), torch.ones(latent_dim))
        prior_mu = torch.tensor([torch.mean(torch.tensor(pomdp_env.stretch_range)),
                    torch.mean(torch.tensor(pomdp_env.shear_range)),
                    torch.mean(torch.tensor(pomdp_env.shear_range)),
                    torch.mean(torch.tensor(pomdp_env.stretch_range))])
        prior_sigma = torch.tensor([.5,.5,.5,.5])
        self.prior = dist.Normal(prior_mu, prior_sigma)

        
    def sample_latent(self, n_samples=1):
        """Sample from the variational distribution q(theta)"""
        eps = torch.randn(n_samples, self.latent_dim)
        std = torch.exp(0.5 * self.q_logvar)
        return self.q_mu + eps * std
    
    def elbo(self, X, y, n_samples=10):
        """
        Compute the evidence lower bound (ELBO)
        
        Args:
            X: torch.Tensor, input data (batch_size, input_dim)
            y: torch.Tensor, observations (batch_size,)
            n_samples: int, number of Monte Carlo samples
        """
        batch_size = X.shape[0]
        
        # Sample from variational distribution
        theta_samples = self.sample_latent(n_samples)  # (n_samples, latent_dim)
        
        # Compute log likelihood for each sample
        log_likelihood = torch.zeros(n_samples, batch_size)
        for i in range(n_samples):
            theta = theta_samples[i]
            y_pred = self.f(X, theta.repeat(batch_size, 1))
            # Assuming Bernoulli observation model
            log_likelihood[i] = dist.Bernoulli(y_pred.squeeze()).log_prob(y)
        
        # Average over samples
        expected_log_likelihood = torch.mean(log_likelihood, dim=0).sum()

        # if expected_log_likelihood == dist.Bernoulli(self.f(X,theta_samples)).log_prob(y).sum():
        #     print('ok should substitute the for loop with the above line')
        
        # Compute KL divergence
        q_dist = dist.Normal(self.q_mu, torch.exp(0.5 * self.q_logvar))
        kl_div = dist.kl_divergence(q_dist, self.prior).sum()
        
        return expected_log_likelihood - kl_div
    
    def fit(self, X, y, n_epochs=100, batch_size=64, lr=0.1):
        """
        Fit the model using variational inference
        
        Args:
            X: torch.Tensor, input data
            y: torch.Tensor, observations
            n_epochs: int, number of training epochs
            batch_size: int, batch size for stochastic optimization
            lr: float, learning rate
        """
        optimizer = Adam([self.q_mu, self.q_logvar], lr=lr)
        
        dataset = torch.utils.data.TensorDataset(X, y)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        for epoch in range(n_epochs):
            epoch_loss = 0
            for batch_X, batch_y in dataloader:
                optimizer.zero_grad()
                loss = -self.elbo(batch_X, batch_y,100)  # Negative because we minimize
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            
            if (epoch + 1) % 100 == 0:
                print(f"Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss/len(dataloader):.4f}")
    
    def get_posterior_params(self):
        """Return the learned posterior parameters"""
        return {
            'mean': self.q_mu.detach(),
            'std': torch.exp(0.5 * self.q_logvar).detach()
        }

In [72]:
# X va sostituito con gli stati del pomdp (x,y)
X = torch.stack(positions)

# y va sostituito con le osservazioni del pomdp negli stati pos=(x.y)
y = torch.stack(observations)

# true_theta va sostituito con i parametri reali del pomdp
true_theta = pomdp_env.transformation_matrix 

# Create and fit the model
model = VariationalBayesianInference(obs_model, input_dim=2, latent_dim=4)
model.fit(X, y)

# Get the learned posterior parameters
posterior = model.get_posterior_params()
print("Estimated theta (mean):", posterior['mean'])
print("Posterior standard deviation:", posterior['std'])

print("True theta:", true_theta)

Epoch 100/1000, Loss: 173.8288
Epoch 200/1000, Loss: 46.3040
Epoch 300/1000, Loss: 28.8903
Epoch 400/1000, Loss: 122.4874
Epoch 500/1000, Loss: 28.5610
Epoch 600/1000, Loss: 29.0916
Epoch 700/1000, Loss: 29.2610
Epoch 800/1000, Loss: 28.3988
Epoch 900/1000, Loss: 28.7424
Epoch 1000/1000, Loss: 28.7606
Estimated theta (mean): tensor([ 3.0437, -1.1561, -0.1792,  0.7987])
Posterior standard deviation: tensor([0.0325, 0.0114, 0.0087, 0.0062])
True theta: [[0.5079403171435333, 0.17129979562961556], [-0.10487444801468786, 0.8962749936302412]]


In [70]:
# check everything is working
theta = torch.tensor(true_theta).flatten()
theta = torch.tensor([[.5,0.2,-0.1,.9]])
y_pred = obs_model(X, theta.repeat(X.shape[0], 1))
# # Assuming Bernoulli observation model
log_likelihood = dist.Bernoulli(y_pred.squeeze()).log_prob(y).sum()
log_likelihood


tensor(-0.7534, grad_fn=<SumBackward0>)

In [73]:
pomdp_env.render()

(None, None, None, None, None)

In [52]:
log_likelihood

tensor(-0.1250, grad_fn=<SumBackward0>)

# metropolis

In [35]:
import torch
import torch.distributions as dist

# Define the multivariate Gaussian prior: N(mu, Sigma)
def prior(theta, mu, sigma):
    """
    theta: Tensor of shape (2,) representing [theta1, theta2].
    mu: Tensor of shape (2,) representing [mu1, mu2].
    sigma: Tensor of shape (2, 2), covariance matrix.
    """
    mvn = dist.MultivariateNormal(mu, sigma)
    return mvn.log_prob(theta)

def likelihood(X,theta,y):
    """
    y: observation
    X: position of the agent
    theta: deformation parameter (sampled from the variational distribution)
    
    Returns the log likelihood of the observation y given the position X and the deformation parameter theta
    """
    # print(X.shape, theta.shape)
    f_theta = obs_model(X, theta.repeat(X.shape[0],1))
    return y * torch.log(f_theta + 10e-9) + (1 - y) * torch.log(1 - f_theta + 10e-9)


# Define the posterior (up to proportionality)
def log_posterior(X,theta, y, mu, sigma):
    """
    theta: Tensor of shape (2,) representing [theta1, theta2].
    """
    return likelihood(X,theta, y).sum() + prior(theta, mu, sigma)

# Metropolis-Hastings algorithm
def metropolis_hastings(X,y, num_samples=10000, burn_in=1000, mu=None, sigma=None, proposal_std=0.5):
    """
    y: Observed binary outcomes (0 or 1).
    num_samples: Number of samples to draw from posterior.
    burn_in: Number of initial samples to discard.
    mu: Tensor of shape (2,) representing the prior mean vector.
    sigma: Tensor of shape (2, 2), the prior covariance matrix.
    proposal_std: Standard deviation for the proposal distribution.
    """
    if mu is None:
        raise ValueError("Prior mean (mu) must be specified.")
        mu = torch.tensor([0.0, 0.0])
    if sigma is None:
        sigma = torch.eye(2)  # Identity covariance matrix by default

    samples = []
    theta = torch.zeros(4)  # Initial value for [theta1, theta2]
    current_log_posterior = log_posterior(X,theta, y, mu, sigma)

    # Define proposal distribution
    proposal_cov = torch.eye(4) * proposal_std**2

    for i in range(num_samples + burn_in):
        # Propose new theta' using a multivariate normal proposal
        proposal = dist.MultivariateNormal(theta, proposal_cov).sample()

        # Compute the log-posterior for the proposal
        proposed_log_posterior = log_posterior(X,proposal, y, mu, sigma)

        # Compute acceptance probability
        acceptance_prob = torch.exp(proposed_log_posterior - current_log_posterior)

        # Accept or reject the proposal
        if torch.rand(1).item() < acceptance_prob:
            theta = proposal
            current_log_posterior = proposed_log_posterior

        # Store the sample after burn-in
        if i >= burn_in:
            samples.append(theta.numpy())

    return torch.tensor(samples)

# Example usage
X = torch.stack(positions)
# Observations (y): binary data from a Bernoulli distribution
y = torch.stack(observations)

# Define prior parameters
mu = torch.tensor(torch.tensor([torch.mean(torch.tensor(pomdp_env.stretch_range)),
                torch.mean(torch.tensor(pomdp_env.shear_range)),
                torch.mean(torch.tensor(pomdp_env.shear_range)),
                torch.mean(torch.tensor(pomdp_env.stretch_range))]))
sigma = torch.eye(4) * 0.2  # Identity covariance matrix

# Run Metropolis-Hastings
samples = metropolis_hastings(X,y, num_samples=5000, burn_in=1000, mu=mu, sigma=sigma, proposal_std=0.5)

# Analyze and visualize results
import matplotlib.pyplot as plt

# Extract theta1 and theta2 samples
theta1_samples = samples[:, 0].numpy()
theta2_samples = samples[:, 1].numpy()
theta3_samples = samples[:, 2].numpy()
theta4_samples = samples[:, 3].numpy()

# Plot the posterior samples
plt.figure(figsize=(10, 5))

# Theta1 histogram
plt.subplot(1, 2, 1)
plt.hist(theta1_samples, bins=50, density=True, alpha=0.7, label='Theta1 Samples')
plt.title("Posterior Distribution of Theta1")
plt.xlabel("Theta1")
plt.ylabel("Density")
plt.legend()

# Theta2 histogram
plt.subplot(1, 2, 2)
plt.hist(theta2_samples, bins=50, density=True, alpha=0.7, label='Theta2 Samples')
plt.title("Posterior Distribution of Theta2")
plt.xlabel("Theta2")
plt.ylabel("Density")
plt.legend()

# plt.subplot(1, 2, 1)
# plt.hist(theta3_samples, bins=50, density=True, alpha=0.7, label='Theta3 Samples')
# plt.title("Posterior Distribution of Theta3")
# plt.xlabel("Theta3")
# plt.ylabel("Density")
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.hist(theta4_samples, bins=50, density=True, alpha=0.7, label='Theta4 Samples')
# plt.title("Posterior Distribution of Theta4")
# plt.xlabel("Theta4")
# plt.ylabel("Density")
# plt.legend()


plt.tight_layout()
plt.show()


/tmp/ipykernel_5124/741569895.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mu = torch.tensor(torch.tensor([torch.mean(torch.tensor(pomdp_env.stretch_range)),


KeyboardInterrupt: 

In [45]:
pomdp_env.transformation_matrix

[[0.435245612516588, -0.19779374326553678],
 [-0.19766264973518238, 0.5614838793112605]]